# Analysis and Visualization

This notebook creates visualizations and prepares data for the analysis report.

Includes:
- Learning curves from training
- Evaluation metrics
- Performance analysis
- Comparison with baseline (HMM-only agent)


In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load results
with open('../results/training_history.pkl', 'rb') as f:
    training_history = pickle.load(f)

with open('../results/evaluation_results.pkl', 'rb') as f:
    evaluation_results = pickle.load(f)

print("Results loaded")


In [ ]:
# Summary statistics
print("=" * 60)
print("FINAL RESULTS SUMMARY")
print("=" * 60)
print(f"Success Rate: {evaluation_results['success_rate']:.4f} ({evaluation_results['success_rate']*100:.2f}%)")
print(f"Total Wrong Guesses: {evaluation_results['total_wrong_guesses']}")
print(f"Total Repeated Guesses: {evaluation_results['total_repeated_guesses']}")
print(f"Final Score: {evaluation_results['final_score']:.2f}")
print("=" * 60)


In [ ]:
# Comprehensive visualization
fig = plt.figure(figsize=(18, 12))

# 1. Training: Win Rate
ax1 = plt.subplot(2, 3, 1)
window = 100
if len(training_history['episode_wins']) >= window:
    wins_array = np.array(training_history['episode_wins'])
    moving_avg = np.convolve(wins_array, np.ones(window)/window, mode='valid')
    ax1.plot(range(window-1, len(wins_array)), moving_avg, linewidth=2)
    ax1.set_title('Training: Win Rate Over Time', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Episode')
    ax1.set_ylabel('Win Rate')
    ax1.grid(True, alpha=0.3)

# 2. Training: Episode Rewards
ax2 = plt.subplot(2, 3, 2)
rewards_array = np.array(training_history['episode_rewards'])
moving_avg_rewards = np.convolve(rewards_array, np.ones(window)/window, mode='valid')
ax2.plot(range(window-1, len(rewards_array)), moving_avg_rewards, linewidth=2, color='green')
ax2.set_title('Training: Episode Rewards', fontsize=12, fontweight='bold')
ax2.set_xlabel('Episode')
ax2.set_ylabel('Average Reward')
ax2.grid(True, alpha=0.3)

# 3. Training: Wrong Guesses
ax3 = plt.subplot(2, 3, 3)
wrong_array = np.array(training_history['episode_wrong_guesses'])
moving_avg_wrong = np.convolve(wrong_array, np.ones(window)/window, mode='valid')
ax3.plot(range(window-1, len(wrong_array)), moving_avg_wrong, linewidth=2, color='red')
ax3.set_title('Training: Wrong Guesses Over Time', fontsize=12, fontweight='bold')
ax3.set_xlabel('Episode')
ax3.set_ylabel('Average Wrong Guesses')
ax3.grid(True, alpha=0.3)

# 4. Epsilon Decay
ax4 = plt.subplot(2, 3, 4)
ax4.plot(training_history['epsilon_values'], linewidth=2, color='purple')
ax4.set_title('Exploration Rate (Epsilon) Decay', fontsize=12, fontweight='bold')
ax4.set_xlabel('Episode')
ax4.set_ylabel('Epsilon')
ax4.grid(True, alpha=0.3)

# 5. Evaluation Progress
ax5 = plt.subplot(2, 3, 5)
if training_history['eval_win_rates']:
    eval_episodes = [500 * (i+1) for i in range(len(training_history['eval_win_rates']))]
    ax5.plot(eval_episodes, training_history['eval_win_rates'], marker='o', linewidth=2, markersize=8)
    ax5.set_title('Evaluation Win Rate During Training', fontsize=12, fontweight='bold')
    ax5.set_xlabel('Training Episode')
    ax5.set_ylabel('Win Rate')
    ax5.grid(True, alpha=0.3)

# 6. Final Evaluation Metrics
ax6 = plt.subplot(2, 3, 6)
metrics = ['Success\nRate', 'Wrong\nGuesses\n(per game)', 'Repeated\nGuesses\n(per game)']
values = [
    evaluation_results['success_rate'],
    evaluation_results['total_wrong_guesses'] / evaluation_results['num_games'],
    evaluation_results['total_repeated_guesses'] / evaluation_results['num_games']
]
bars = ax6.bar(metrics, values, color=['green', 'red', 'orange'], alpha=0.7, edgecolor='black', linewidth=1.5)
ax6.set_title('Final Evaluation Metrics', fontsize=12, fontweight='bold')
ax6.set_ylabel('Value')
ax6.grid(True, alpha=0.3, axis='y')
# Add value labels on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.3f}',
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('../results/comprehensive_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Comprehensive visualization saved")


In [ ]:
# Performance by word length
detailed_results = evaluation_results['detailed_results']
df = pd.DataFrame(detailed_results)
df['word_length'] = df['word'].apply(len)

# Group by word length
length_stats = df.groupby('word_length').agg({
    'won': ['sum', 'count', 'mean'],
    'wrong_guesses': 'mean',
    'repeated_guesses': 'mean'
}).round(3)

print("\nPerformance by Word Length:")
print("=" * 80)
print(length_stats)

# Visualize performance by word length
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Win rate by length
length_win_rate = df.groupby('word_length')['won'].mean()
axes[0].bar(length_win_rate.index, length_win_rate.values, alpha=0.7, edgecolor='black')
axes[0].set_title('Win Rate by Word Length', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Word Length')
axes[0].set_ylabel('Win Rate')
axes[0].grid(True, alpha=0.3, axis='y')

# Average wrong guesses by length
length_wrong = df.groupby('word_length')['wrong_guesses'].mean()
axes[1].bar(length_wrong.index, length_wrong.values, alpha=0.7, color='red', edgecolor='black')
axes[1].set_title('Average Wrong Guesses by Word Length', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Word Length')
axes[1].set_ylabel('Average Wrong Guesses')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../results/performance_by_length.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Performance by length visualization saved")


In [ ]:
# Generate Analysis Report PDF
import sys
sys.path.append('../src')
from generate_report import generate_report

print("Generating Analysis Report PDF...")
try:
    generate_report()
    print("✓ Analysis Report PDF generated successfully!")
except Exception as e:
    print(f"Error generating PDF: {e}")
    print("Note: You may need to install reportlab: pip install reportlab")
    print("Alternatively, you can create the PDF manually using the information above.")
